In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import packages
from vcodatools.db.conn import StarburstConnection

import trino


from sqlalchemy import text

# Show all the columns when displaying a dataframe
pd.set_option("display.max_columns", None)

# database here is Starburst Catalog. Can be set to None and specified in query text.
sb = StarburstConnection(database=None)
con = sb.engine.connect()

In [17]:
# database here is Starburst Catalog. Can be set to None and specified in query text.
sb = StarburstConnection(database=None)
con = sb.engine.connect()

In [27]:
# Query for data

query = """
WITH expenditures AS (
  SELECT 
    expenditure_item_date, 
    date_record_created, 
    project_name, 
    project_number, 
    task_name, 
    task_number, 
    incurred_org, 
    expenditure_type_code, 
    expenditure_category,
    regular_time, 
    raw_cost, 
    raw_cost_rate, 
    expenditure_comment, 
    adjusted_exp_flag, 
    bill_hold_flag, 
    billable_flag, 
    billed_flag, 
    cross_charge_flag, 
    cost_distributed_flag, 
    cost_burden_dist_flag, 
    transfer_from_project_flag, 
    labor_or_nonlabor_flag
  FROM 
    oracle.oracle_views.all_expenditure_items
  WHERE 
    project_number IN ('0001929', '0001899', '0001949')
    AND labor_or_nonlabor_flag = 'Y'
)

SELECT 
  neg.date_record_created,
  neg.task_name,
  neg.task_number,
  neg.raw_cost AS neg_raw_cost,
  neg.regular_time AS neg_regular_time,
  pos.raw_cost AS pos_raw_cost,
  pos.regular_time AS pos_regular_time
FROM 
  expenditures AS neg
LEFT JOIN 
  expenditures AS pos
ON 
  neg.date_record_created = pos.date_record_created
  AND neg.task_name = pos.task_name
  AND neg.task_number = pos.task_number
  AND neg.raw_cost = -pos.raw_cost
  AND neg.regular_time = -pos.regular_time
WHERE 
  neg.raw_cost < 0
ORDER BY 
  neg.date_record_created, neg.task_number, neg.task_name, pos.raw_cost
"""

# Query and store in datetime-indexed data frame
df = pd.read_sql_query(sql=text(query), con=con)

In [28]:
df.shape

(19126, 7)

In [29]:
df

,date_record_created,task_name,task_number,neg_raw_cost,neg_regular_time,pos_raw_cost,pos_regular_time
0,2019-05-10 21:38:35,vGate Mgmt Reviews,1.04.01.02,-71.87,-1.5,71.87,1.5
1,2019-05-12 21:36:54,vGate Mgmt Reviews,1.04.01.02,-74.04,-1.0,74.04,1.0
2,2019-05-12 21:36:54,vGate Mgmt Reviews,1.04.01.02,-102.05,-1.5,102.05,1.5
3,2019-05-12 21:36:54,vGate Mgmt Reviews,1.04.01.02,-111.06,-1.5,111.06,1.5
4,2019-05-12 21:36:54,Finance Mgmt Lead,1.04.04x.01,-296.16,-4.0,296.16,4.0
...,...,...,...,...,...,...,...
19121,2024-06-03 23:32:17,13m IOC Install&SAT,1.10.01.01.02,-54.64,-2.0,NaN,NaN
19122,2024-06-03 23:32:17,13m IOC Install&SAT,1.10.01.01.02,-54.64,-2.0,NaN,NaN
19123,2024-06-03 23:32:18,Operations Mgmt,1.04.06.01.01,-19.95,-0.5,NaN,NaN
19124,2024-06-03 23:32:18,Operations Mgmt,1.04.06.01.01,-39.90,-1.0,NaN,NaN
